In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-135156")
exp = Experiment(workspace=ws, name="quick-starts-ws-135156")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135156
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-135156


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "hyperdriveml"
cpu_cluster = ComputeTarget(ws,cpu_cluster_name)

try:
       cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
       print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
%%writefile conda_dependencies.yml

dependencies:
  - python=3.6.2
  - scikit-learn
  - numpy
  - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [4]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({"max_iter" : choice(100,150,200,250),
                                "C" : choice(0.01,0.05,0.2,1,5,10)
                                })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target=cpu_cluster,                      
                      environment=sklearn_env)


# est = SKLearn(source_directory='./',               
#             compute_target=cpu_cluster,
#             entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hd_config = HyperDriveConfig(run_config=src,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='Accuracy',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=20,
                                max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
#experimnt = Experiment(ws,exp)
hd_run = exp.submit(hd_config)
hd_run.wait_for_completion(show_output=True)
RunDetails(hd_run).show()
### YOUR CODE HERE ###

RunId: HD_38976887-4b4e-44a9-b8a3-7505fe62f988
Web View: https://ml.azure.com/experiments/quick-starts-ws-135156/runs/HD_38976887-4b4e-44a9-b8a3-7505fe62f988?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-135156/workspaces/quick-starts-ws-135156

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-18T05:26:11.362922][API][INFO]Experiment created<END>\n""<START>[2021-01-18T05:26:11.840672][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-18T05:26:12.030101][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-18T05:26:12.5878026Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_38976887-4b4e-44a9-b8a3-7505fe62f988
Web View: https://ml.azure.com/experiments/quick-starts-ws-135156/runs/HD_38976887-4b4e-44a9-b8a3-7505fe62f988?wsid=/subsc

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
import joblib
# Get your best run and save the model from that run.
### YOUR CODE HERE ###
best_run = hd_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-135156,HD_38976887-4b4e-44a9-b8a3-7505fe62f988_3,azureml.scriptrun,Failed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
print("Best run details", best_run.get_details())
print('\t')
print("Best run file names:", best_run.get_file_names())
model = best_run.register_model(model_name='bankmark_hyperdrive_best_run', model_path='outputs/model.joblib')
print(model.name, model.id,model.version,sep='\t')

Best run details {'runId': 'HD_38976887-4b4e-44a9-b8a3-7505fe62f988_3', 'target': 'hyperdriveml', 'status': 'Failed', 'startTimeUtc': '2021-01-18T05:26:53.909899Z', 'endTimeUtc': '2021-01-18T05:29:27.47102Z', 'error': {'error': {'code': 'UserError', 'message': 'AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code', 'messageFormat': '{Message}', 'messageParameters': {'Message': 'AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code'}, 'details': [], 'innerError': {'code': 'BadArgument', 'innerError': {'code': 'AmlComputeBadRequest'}}}, 'correlation': {'operation': None, 'request': 'dcfecce529f1c5d4'}, 'environment': 'southcentralus', 'location': 'southcentralus', 'time': '2021-01-18T05:29:27.295604Z', 'componentName': 'execution-worker'}, 'properties':

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_1f98e8aa5eed21a822a6aa9678e903ccd108a2c1a633666aaf33ac91da55e6de_d.txt', 'azureml-logs/65_job_prep-tvmps_1f98e8aa5eed21a822a6aa9678e903ccd108a2c1a633666aaf33ac91da55e6de_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_1f98e8aa5eed21a822a6aa9678e903ccd108a2c1a633666aaf33ac91da55e6de_d.txt', 'logs/azureml/93_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_a972eb25-7356-483d-95ea-3cddcd0dfbba.jsonl', 'logs/azureml/dataprep/python_span_l_a972eb25-7356-483d-95ea-3cddcd0dfbba.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorRes

In [ ]:
hd_run.get_metrics()

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.core.dataset import Dataset

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds= TabularDatasetFactory.from_delimeted_files(path=data_url)


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x['y'] = y
label = "y"

In [ ]:
x.head()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name=label,
    n_cross_validations=3)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
#ws = Workspace.from_config()
#experiment = Experiment(ws, "exp")
run = exp.submit(automl_config, show_output=True)
run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
from azureml.interpret import ExplanationClient

# Get model explanation data
client = ExplanationClient.from_run(best_run)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [ ]:
client = ExplanationClient.from_run_id(ws, experiment_name, best_run.id)
global_explanation = client.download_model_explanation()
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [ ]:
global_explanation_topk = client.download_model_explanation(top_k=4)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [ ]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

In [ ]:
model_name = best_run.properties['model_name']

script_file_name = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')